# Web Scraping IMDB Top Rated Movies:

## Shreya Kumar (861279837)

In [55]:
# importing basic packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# packages for web scraping
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from requests import get
import re

In [56]:
#  URL containing dataset
link1 = "https://www.imdb.com/chart/top?ref_=nv_mv_250"
main_url = link1
html = urlopen(main_url)
msoup = BeautifulSoup(html, 'lxml')

# Movies by title
moviesList = msoup.find_all('td', class_="titleColumn")
#testing
#print (len(moviesList))

In [57]:
# movie names
movieName = []
for row in moviesList:
    # title list
    mn = row.findAll('a')
    movieName.append(mn[0].text)

# year released
yearReleased = []
for row in moviesList:
    # year release list
    yr = row.findAll('span')
    strip1= yr[0].text
    strip2= strip1.strip('(')
    strip3= strip2.strip(')')
    yearReleased.append(float(strip3))

# links for individual movies
movieLinks = []
for row in moviesList:
    # movie links list
    lnk = row.findAll('a', attrs={'href': re.compile("/title")})
    movieLinks.append(lnk[0].get('href')) 

# movie ratings
imdbRate = []
imdbRating = msoup.find_all('td', class_="ratingColumn imdbRating")
for row in imdbRating:
    rt = row.findAll('strong')
    imdbRate.append(float(rt[0].text))
    

In [58]:
# data from of movie lists from pages
df = pd.DataFrame({'Name of the movie': movieName, 
                   'Link':movieLinks, 
                   'Year released' :yearReleased, 
                   'IMDB rating': imdbRate})

In [59]:
#testing
#df.head(5)

In [75]:
#values for individual movies
revList = [] # list of reviewers
movieRating = [] # movie rating
movieLength=[] # movie length
genre=[] # genre
relDate=[] # release date
summary=[] # movie summary
director=[] # movie director
writer=[] # movie writers
stars=[] # movie stars
keyword=[] # keywords
budget=[] # budget
grossUS=[] # Gross USA
prodCmp=[] # Production company

# loops through top 250 movie pages
for i in range (0, len(moviesList)): 
    mov_url = "https://www.imdb.com"+df.Link[i]
    mov_html = urlopen(mov_url)
    mvsoup = BeautifulSoup(mov_html, 'lxml')
    
    # movie reviewers
    reviewers = mvsoup.find_all('span', itemprop="ratingCount" )
    res1 = re.sub(',','',reviewers[0].text)
    revList.append(float(res1))
    
    # movie rating
    ratings = mvsoup.find('div', class_="subtext").contents
    res1 = re.sub(re.compile('\n'), '',ratings[0].string)
    res2 = res1.strip()
    movieRating.append(res2) 
    
    # movie length
    res3 = mvsoup.find('time').string
    res4 = re.sub(re.compile('\n'), '',res3)
    duration = res4.strip()
    movieLength.append(duration)
    
    # movie genre
    gen1 = mvsoup.find_all('div', class_="see-more inline canwrap")
    cnt=0
    for j in range (0,len(gen1)):
        if gen1[j].h4 is not None:
            res1 = gen1[j].h4.string
            if res1=="Genres:":
                res21=gen1[j].text
                genre.append(res21)
                cnt=1
        if j==len(gen1)-1 and cnt==0:
            genre.append(" ")
    
    # movie release date
    res5 = mvsoup.find('a', title="See more release dates").string
    rldt = re.sub(re.compile('\n'), '',res5)
    relDate.append(rldt)
    
    # movie summary 
    res6 = mvsoup.find('div', class_="summary_text").string
    if res6 is not None:
        res7 = re.sub(re.compile('\n'), '',res6)
        smmry = res7.strip()
        summary.append(smmry)
    else :
        summary.append(" ")
    
    
    # movie director
    credt = mvsoup.find_all('div', class_="credit_summary_item")
    cnt=0
    for j in range (0,len(credt)):
        res1 = credt[j].h4.string
        if res1=="Director:":
            res21=credt[j].text
            director.append(res21)
            cnt=1
        if j==len(credt)-1 and cnt==0:
            director.append(" ")
    
    # movie writers
    for j in range (0,3):
        res1 = credt[j].h4.string
        if res1== 'Writer:' or res1== 'Writers:':
            res21=credt[j].text
            writer.append(res21)
    
    # movie stars
    for j in range (0,3):
        res1 = credt[j].h4.string
        if res1== 'Stars:' or res1== 'Star:':
            res21=credt[j].text
            stars.append(res21)
    
    # keywords
    key = mvsoup.find_all('div', class_="see-more inline canwrap")
    cnt=0
    for j in range (0,len(key)):
        if key[j].h4 is not None:
            res1 = key[j].h4.string
            if res1=="Plot Keywords:":
                res21=key[j].text
                keyword.append(res21)
                cnt=1
        if j==len(key)-1 and cnt==0:
            keyword.append(" ")
            
    
    # movie budget
    txtblk = mvsoup.find_all('div', class_="txt-block")
    cntr=0
    for j in range (6,len(txtblk)):
        if txtblk[j].h4 is not None:        
            res1 = txtblk[j].h4.string
            if res1=="Budget:":
                res21=txtblk[j].text
                cntr=1
                budget.append(res21)
        if j==len(txtblk)-1 and cntr==0:
            budget.append(" ")
    
    # movie gross USA
    cntr=0
    for j in range (6,len(txtblk)):
        if txtblk[j].h4 is not None:        
            res1 = txtblk[j].h4.string
            if res1=="Gross USA:":
                res21=txtblk[j].text
                cntr=1
                grossUS.append(res21)
        if j==len(txtblk)-1 and cntr==0:
            grossUS.append(" ")
    
    # production company
    cnt=0
    for j in range (8,len(txtblk)):
        if txtblk[j].h4 is not None:        
            res1 = txtblk[j].h4.string
            if res1=="Production Co:":
                res21=txtblk[j].text
                cnt=1
                prodCmp.append(res21)
        if j==len(txtblk)-1 and cnt==0:
            prodCmp.append(" ")


In [61]:
df2 = pd.DataFrame({'Reviewers': revList,
                   'Movie_rating':movieRating, 
                   'Movie_duration' :movieLength, 
                   'Genre': genre,
                   'Release_date': relDate,
                   'Summary': summary,
                   'Director': director,
                   'Writers': writer,
                   'Stars': stars,
                   'PlotKeywords':keyword,
                   'Budget': budget,
                   'Gross_USA': grossUS,
                   'Production_Company': prodCmp})

In [62]:
#testing
#df2.head(5)

In [63]:
# clean genre
for i in range(0,len(genre)): 
    comp = re.compile(r'(Genres:|\n|\xa0)')
    strip1 = re.sub(comp,'',genre[i])
    comp1 = re.compile(r'\|')
    strip2 = re.sub(comp1,',',strip1)
    strip3=strip2.strip()
    genre[i] = strip3

In [64]:
# splitting genre
genre1=[]
genre2=[]
genre3=[]
genre4=[]
for i in range (0, len(genre)):
    split= re.split(',', genre[i])
    if len(split)==1:
        genre1.append(split[0])
        genre2.append(" ")
        genre3.append(" ") 
        genre4.append(" ")
    if len(split)>1 and len(split)<3:
        genre1.append(split[0])
        genre2.append(split[1])
        genre3.append(" ") 
        genre4.append(" ")
    if len(split)>2 and len(split)<4:
        genre1.append(split[0])
        genre2.append(split[1])
        genre3.append(split[2]) 
        genre4.append(" ")
    if len(split)>3 and len(split)<5:
        genre1.append(split[0])
        genre2.append(split[1])
        genre3.append(split[2]) 
        genre4.append(split[3])
    if len(split)>=5:
        genre1.append(split[0])
        genre2.append(split[1])
        genre3.append(split[2]) 
        genre4.append(split[3])


In [65]:
# clean directors list
for i in range(0,len(director)): 
    comp=re.compile(r'(Director:|\n)')
    strip1=re.sub(comp, '',director[i])
    strip2=strip1.strip()
    director[i] = strip2

In [66]:
# clean writers list
for i in range(0,len(writer)): 
    comp=re.compile(r'(Writer:|Writers:|\n|more credit\xa0»|more credits\xa0»)')
    strip1=re.sub(comp,'',writer[i])
    comp2=re.compile(r'\|4 | \|3 | \|2 | \|1')
    strip2=re.sub(comp2,'',strip1)
    strip3=strip2.strip()
    writer[i]=strip3

In [67]:
# split writers
writer1=[]
writer2=[]
writer3=[]
for i in range (0, len(writer)):
    split= re.split(',', writer[i])
    if len(split)==1:
        writer1.append(split[0].strip())
        writer2.append(" ")
        writer3.append(" ")       
    if len(split)>1 and len(split)<3:
        writer1.append(split[0].strip())
        writer2.append(split[1].strip())
        writer3.append(" ")
    if len(split)>2:
        writer1.append(split[0].strip())
        writer2.append(split[1].strip())
        writer3.append(split[2].strip()) 

In [68]:
# clean stars
for i in range(0,len(stars)): 
    comp=re.compile(r'(Stars:|\n|See full cast & crew\xa0»)')
    strip1=re.sub(comp,'',stars[i])
    comp2=re.compile(r'\|')
    strip2=re.sub(comp2,'',strip1)
    strip3=strip2.strip()
    stars[i]=strip3

In [69]:
# split stars
star1=[]
star2=[]
star3=[]
star4=[]
star5=[]

for i in range (0, len(stars)):
    split = re.split(',', stars[i])
    if len(split)==1:
        star1.append(split[0])
        star2.append(" ")
        star3.append(" ") 
        star4.append(" ")
        star5.append(" ")
        
    if len(split)>1 and len(split)<3:
        star1.append(split[0])
        star2.append(split[1])
        star3.append(" ") 
        star4.append(" ")
        star5.append(" ")
        
    if len(split)>2 and len(split)<4:
        star1.append(split[0])
        star2.append(split[1])
        star3.append(split[2]) 
        star4.append(" ")
        star5.append(" ")
        
    if len(split)>3 and len(split)<5:
        star1.append(split[0])
        star2.append(split[1])
        star3.append(split[2]) 
        star4.append(split[3])
        star5.append(" ")
        
    if len(split)>4 and len(split)<6:
        star1.append(split[0])
        star2.append(split[1])
        star3.append(split[2]) 
        star4.append(split[3])
        star5.append(split[4])

In [70]:
# clean keywords
for i in range(0,len(keyword)): 
    comp=re.compile(r'(Plot Keywords:|\n|\xa0See All|\xa0»)')
    strip1=re.sub(comp,'',keyword[i])
    comp2=re.compile(r'((|))')
    strip2=re.sub(comp2,'',strip1)
    strip3=strip2.strip()
    keyword[i]=strip3

In [71]:
# clean budget
for i in range(0,len(budget)): 
    comp=re.compile(r'(Budget:|\n|\()|\)|(estimated)')
    strip1=re.sub(comp,'',budget[i])
    strip2=strip1.strip()        
    budget[i]=strip2

# replace symbols
for i in range(0,len(budget)): 
    curr1=budget[i].replace('$', 'USD ')
    curr2=curr1.replace('€', 'EUR ')
    curr3=curr2.replace('£', 'GBP ')
    budget[i]=curr3

# separate currencies
budget_curr=[]
for i in range(0,len(budget)):
    strip1= budget[i][0:3]
    budget_curr.append(strip1)
    

# grab numerical value
budget_val=[]
for i in range(0,len(budget)):
    strip1 = re.sub(',','',budget[i])
    ln = len(strip1)
    if ln>3:
        strip2= strip1[4:ln]
    if ln<3:
        strip2="0"
    budget_val.append(float(strip2))
    

In [72]:
# clean grossUS
for i in range(0,len(grossUS)): 
    comp=re.compile(r'(Gross USA:|\n)')
    strip1=re.sub(comp,'',grossUS[i])
    com1=re.compile(r'\xa0')
    strip0=re.sub(com1,'',strip1)
    strip2=strip0.strip()
    grossUS[i]=strip2
    
# replace symbols
for i in range(0,len(grossUS)): 
    curr1=grossUS[i].replace('$', 'USD ')
    curr2=curr1.replace('€', 'EUR ')
    curr3=curr2.replace('£', 'GBP ')
    grossUS[i]=curr3


for i in range(0,len(grossUS)): 
    comp=re.compile(r'\xa0')
    strip1=re.sub(comp,' ',grossUS[i])
    grossUS[i] = strip1


# separate currencies
grossUS_curr=[]
for i in range(0,len(grossUS)):
    strip1= grossUS[i][0:3]
    grossUS_curr.append(strip1)

# grab numerical value
grossUS_val=[]
for i in range(0,len(grossUS)):
    strip1 = re.sub(',','',grossUS[i])
    ln = len(strip1)
    if ln>2:
        strip2= strip1[4:ln]
    if ln<2:
        strip2='0'
    grossUS_val.append(strip2)


for j in range(0,len(grossUS_val)):
    end = grossUS_val[j].find(' ')
    if end>0:
        strip1 = grossUS_val[j][0:end]
    if end<0:
        strip1 = grossUS_val[j]
    grossUS_val[j]=float(strip1)

    

In [77]:
# clean production company
for i in range(0,len(prodCmp)): 
    comp=re.compile(r'(Production Co:|\n|See more\xa0»)')
    strip1=re.sub(comp,'',prodCmp[i])
    strip2=strip1.strip()
    prodCmp[i]=strip2

In [78]:
#combine all the information
df3 = pd.DataFrame({'Reviewers': revList,
                   'Movie_rating':movieRating, 
                   'Movie_duration' :movieLength, 
                   'Genre1': genre1,
                   'Genre2': genre2,
                   'Genre3': genre3,
                   'Genre4': genre4,
                   'Release_date': relDate,
                   'Summary': summary,
                   'Director': director,
                   'Writer1': writer1,
                   'Writer2': writer2,
                   'Writer3': writer3,
                   'Stars1': star1,
                   'Stars2': star2,
                   'Stars3': star3,
                   'Stars4': star4,
                   'Stars5': star5,
                   'PlotKeywords':keyword,
                   'Budget_curr': budget_curr,
                    'Budget_val': budget_val,
                   'Gross_USA_curr': grossUS_curr,
                    'Gross_USA_val': grossUS_val,
                   'Production_Company': prodCmp})

In [81]:
#testing
#df3.head(5)

In [82]:
#concatenate imdb info with movie info
df4 = pd.concat([df, df3], axis=1, sort=False)

In [85]:
#testing
#df4.head(5)

In [86]:
#produce csv file
df4.to_csv('imdb_dataset.csv', index=False)

----